In [1]:
# IMPORTS

from bs4 import BeautifulSoup
import bs4
import requests
from selenium import webdriver

import pandas as pd
import numpy as np

In [2]:
# USING SPACY FOR WEB SCRAPPING

driver = webdriver.Chrome("chromedriver.exe")

C:\Users\DELL\AppData\Local\Temp/ipykernel_5288/2695741643.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


In [3]:
# LIST OF UNIQUE CATEGORIES THAT WAS NEEDED FOR THE DATASET.
# ADD OR SUBTRACT LABELS AS NEEDED

job_list = ["software developer" , "data analyst" , "software product manager" , 
            "cloud engineer" , "data scientist" , "salesforce developer" , 
            "ios developer" , "database administrator" , "java developer"]

In [4]:
# CREATING A DATAFRAME TO STORE THE LINKS OF EACH INDIVIDUAL RESUME EXAMPLE

resume_links = pd.DataFrame()
category = []
link = []
ids = []
cnt = 0

In [5]:
# LOOP TO SEARCH FOR RESUME EXAMPLES FROM THE WEBPAGE AS PER THE LABELS DEFINED ABOVE. ONLY (10*12) 120 RESUMES WILL BE COLLECTED.

# REMOVE  "&bg=85&eg=100&comp=&mod="  TO EXPAND THE SEARCH 
# (bg=85 & eg=100) FILTERS THE RESUME BASED ON SCORE OF BETWEEEN 85 AND 100, INCREASE RANGE TO EXPAND SEARCH

for job in job_list:
    JOB = job.lower()
    for i in range(1,6):   # INCREASE THE RANGE TO GET MORE RESUME DATA
        PAGE = str(i)
        URL = "https://www.livecareer.com/resume-search/search?jt=" + JOB + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
        driver.get(URL)
        aTagsInLi = driver.find_elements_by_css_selector('li a')
        for a in aTagsInLi:
            if a.get_attribute('rel') == "ugc":
                category.append(JOB)
                link.append(a.get_attribute('href'))
                ids.append(cnt)
                cnt+=1
    

C:\Users\DELL\AppData\Local\Temp/ipykernel_5288/1442043671.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  aTagsInLi = driver.find_elements_by_css_selector('li a')


In [6]:
# STORES THE CATEGORY AND LINK TO THE RESUME WEBPAGE

# resume_links["Category"] = category
# resume_links["link"] = link

In [7]:
# HASHES THE LINK AND CREATES AN UNIQUE ID FOR THE ROWS

# import hashlib
# def id(x):
#     return int(hashlib.md5(x.encode('utf-8')).hexdigest(), 16)

# resume_links["id"] = resume_links["link"].apply(id)

In [8]:
# NUMBER OF RESUME COLLECTED PER DEFINED LABELS

# resume_links.Category.value_counts()

In [9]:
resume_links = pd.DataFrame(columns=['id', 'category', 'resume'])  
# resume_links["Raw_html"] = ""

In [10]:
# LOOP TO COLLECT THE RESUME DATA PRESENT IN THE LINKS AS COLECTED ABOVE
# STORED IN HTML AS WELL AS STRING FORMAT


for i in range(0,len(link)):
    url = link[i]
    driver.get(url)
#     time.sleep(0.5)                  #ADDED DELAY, CAN BE REMOVED
    x = driver.page_source
    x = x.replace(">","> ")
    soup = bs4.BeautifulSoup(x, 'html.parser')
    div = soup.find("div", {"id": "document"})
#     resume_links.Raw_html[i] = div
    try:
        temp = str(div.text)
#         resume_links.Category.append(category[i])
#         resume_links.id.append(ids[i])
#         resume_links.Resume.append(temp)
        resume_links.loc[len(resume_links.index)] = [ids[i], category[i], temp]
    except:
#         ADD EXCEPTION IF REQUIRED
        pass



In [11]:
resume_links.head()

,id,category,resume
0,1,software developer,SOFTWARE DEVELOPER INTERN ...
1,2,software developer,SOFTWARE DEVELOPER Professional...
2,3,software developer,GIS ANALYST/SOFTWARE DEVELOPER ...
3,4,software developer,SENIOR SOFTWARE DEVELOPER Caree...
4,5,software developer,SENIOR SOFTWARE DEVELOPER Summa...


In [12]:
# STORES THE DATAFRAME

resume_links.to_csv("Resume.csv",index=False)